In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SalaDeBatePapoStreaming") \
    .getOrCreate()

In [3]:
df_raw = spark.readStream.format("socket") \
        .option("host", "websocat") \
        .option("port", "9999") \
        .load()


In [12]:
from pyspark.sql.functions import split, col,size
valores = split(col("value")," ")
df_palavras = df_raw.withColumn("palavras", split(col("value")," ")) \
                        .withColumn("qtd_palavras", size(col("palavras")))

In [6]:
from pyspark.sql.functions import expr


In [10]:
palavras_suspeitas = {"ataque", "bomba", "ameaça", "hack", "invasão", "segredo", "explosivo"}


In [8]:
condicao_alerta = expr(" OR ".join([f"array_contains(palavras, '{p}')" for p in palavras_suspeitas]))


In [13]:
df_alertas = df_palavras.filter(condicao_alerta)

In [14]:
df_alertas.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("truncate", False) \
    .start()